In [ ]:
#pip install pandas_datareader

Note: you may need to restart the kernel to use updated packages.


In [25]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

import pandas_datareader.data as web

import os
import glob

In [14]:
# 1) Point to your folder
data_path = "../Data/BTC"  

# 2) Find all files matching btc_2010.csv ... btc_2025.csv
all_files = glob.glob(os.path.join(data_path, "btc_*.csv"))

# 3) Read and collect
df_list = []
for filepath in all_files:
    # adjust parse_dates/index_col to whatever your CSV uses
    year_df = pd.read_csv(
        filepath,
        parse_dates=["timestamp"],    # or the name of your date column
        sep=";"
    )

    df_list.append(year_df)

# 4) Concatenate and sort by date
bitcoin_df = pd.concat(df_list).sort_index()

# 5) (Optional) Reset index if you want Date as a column:
# bitcoin_df = bitcoin_df.reset_index()

bitcoin_df["Date"] = pd.to_datetime(bitcoin_df["timestamp"]).dt.date
bitcoin_df=bitcoin_df.set_index("Date")
bitcoin_df=bitcoin_df.sort_index(ascending=True)

bitcoin_df=bitcoin_df.drop(columns=["timeOpen", "timeClose", "timeHigh", "timeLow", 'name', 'timestamp'])

bitcoin_df.tail()

,open,high,low,close,volume,marketCap
Date,,,,,,
2025-06-07,104390.644797,105972.754088,103987.311793,105615.626309,3.836503e+10,2.099227e+12
2025-06-08,105617.509444,106497.059845,105075.330520,105793.650228,3.662623e+10,2.102767e+12
2025-06-09,105793.021175,110561.419818,105400.230896,110294.099106,5.590319e+10,2.192310e+12
2025-06-10,110295.687653,110380.121446,108367.713433,110257.237712,5.470010e+10,2.191694e+12
2025-06-11,110261.796257,110384.220962,108086.327919,108686.627682,5.084266e+10,2.160417e+12


In [16]:
Main_df=bitcoin_df[['close','volume','marketCap']]
Main_df=Main_df.rename(columns={"close": "BTC", "volume": "BTC_Volume"})
Main_df['BTC_count']=Main_df['marketCap']/Main_df['BTC']
Main_df.drop(columns=['marketCap'], inplace=True)
Main_df.head()

,BTC,BTC_Volume,BTC_count
Date,,,
2010-07-14,0.056402,261.54,3.373269e+06
2010-07-15,0.057568,445.80,3.404354e+06
2010-07-16,0.066492,497.25,3.429713e+06
2010-07-17,0.065993,19.99,3.438340e+06
2010-07-18,0.078814,75.13,3.446975e+06


In [17]:
ETH_df = pd.read_csv("../Data/ethereum.csv", sep=";")
ETH_df["Date"] = pd.to_datetime(ETH_df["timestamp"]).dt.date
ETH_df=ETH_df.set_index("Date")
ETH_df=ETH_df.sort_index(ascending=True)

ETH_df=ETH_df.drop(columns=["timeOpen", "timeClose", "timeHigh", "timeLow", 'name', 'timestamp'])

ETH_df.head()

,open,high,low,close,volume,marketCap
Date,,,,,,
2015-08-08,2.793760,2.798810,0.714725,0.753325,674188.0,4.548689e+07
2015-08-09,0.706136,0.879810,0.629191,0.701897,532170.0,4.239957e+07
2015-08-10,0.713989,0.729854,0.636546,0.708448,405283.0,4.281836e+07
2015-08-11,0.708087,1.131410,0.663235,1.067860,1463100.0,6.456929e+07
2015-08-12,1.058750,1.289940,0.883608,1.217440,2150620.0,7.364501e+07


In [18]:
Main_df["ETH"]=ETH_df["close"]
Main_df["ETH_Volume"]=ETH_df["volume"]
Main_df["ETH_count"]=ETH_df["marketCap"]/ETH_df["close"]
Main_df.tail()

,BTC,BTC_Volume,BTC_count,ETH,ETH_Volume,ETH_count
Date,,,,,,
2025-06-07,105615.626309,3.836503e+10,1.987610e+07,NaN,NaN,NaN
2025-06-08,105793.650228,3.662623e+10,1.987611e+07,NaN,NaN,NaN
2025-06-09,110294.099106,5.590319e+10,1.987695e+07,NaN,NaN,NaN
2025-06-10,110257.237712,5.470010e+10,1.987801e+07,NaN,NaN,NaN
2025-06-11,108686.627682,5.084266e+10,1.987749e+07,NaN,NaN,NaN


In [ ]:
# Broad trade-weighted dollar index, Jan 2006=100
dxy_fred = web.DataReader(
    "DTWEXBGS",      # Broad index (not discontinued)
    "fred",
    start="2010-01-01",
    end="2025-05-04"
)

# Clean up
dxy_fred.columns = ["DXY_broad"]
dxy_fred.sort_index(inplace=True)


dxy_fred.to_csv("../Data/dxy.csv", index=True)

dxy_fred.head()


,DXY_broad
DATE,
2010-01-01,NaN
2010-01-04,92.3566
2010-01-05,92.2236
2010-01-06,92.0941
2010-01-07,92.3684


In [20]:
print(dxy_fred.shape)
dxy_fred.isna().sum()

(4001, 1)


DXY_broad    184
dtype: int64

In [21]:
Main_df["dxy"]=dxy_fred["DXY_broad"]
Main_df['dxy'] = Main_df['dxy'].interpolate(method='linear')
Main_df.tail()

,BTC,BTC_Volume,BTC_count,ETH,ETH_Volume,ETH_count,dxy
Date,,,,,,,
2025-06-07,105615.626309,3.836503e+10,1.987610e+07,NaN,NaN,NaN,122.859
2025-06-08,105793.650228,3.662623e+10,1.987611e+07,NaN,NaN,NaN,122.859
2025-06-09,110294.099106,5.590319e+10,1.987695e+07,NaN,NaN,NaN,122.859
2025-06-10,110257.237712,5.470010e+10,1.987801e+07,NaN,NaN,NaN,122.859
2025-06-11,108686.627682,5.084266e+10,1.987749e+07,NaN,NaN,NaN,122.859


In [ ]:
start, end = "2010-01-01", "2025-05-04"
codes = {
    "US": "M2SL",            # U.S. M2 Money Stock :contentReference[oaicite:0]{index=0}
    "EA": "MYAGM2EZM196N",   # Euro‐Area M2 :contentReference[oaicite:1]{index=1}
    "CN": "MYAGM2CNM189N",   # China M2 :contentReference[oaicite:2]{index=2}
    # …add Japan, U.K., etc. as needed
}

# Fetch each series and align on a common DateTimeIndex
df = pd.DataFrame({
    country: web.DataReader(code, "fred", start, end)[code]
    for country, code in codes.items()
})

df.to_csv("../Data/M2_supply.csv", index=True)
df.head()


,US,EA,CN
DATE,,,
2024-12-01,21441.8,NaN,NaN
2025-01-01,21520.0,NaN,NaN
2025-02-01,21613.8,NaN,NaN
2025-03-01,21706.8,NaN,NaN
2025-04-01,21862.5,NaN,NaN


In [30]:
Main_df['US_M2']=df['US']
Main_df['EA_M2']=df['EA']
Main_df['CN_M2']=df['CN']
Main_df['US_M2'] = Main_df['US_M2'].interpolate(method='linear')
Main_df['EA_M2'] = Main_df['EA_M2'].interpolate(method='linear')
Main_df['CN_M2'] = Main_df['CN_M2'].interpolate(method='linear')
Main_df.tail()


,BTC,BTC_Volume,BTC_count,ETH,ETH_Volume,ETH_count,dxy,US_M2,EA_M2,CN_M2,reward,days_since_halving
Date,,,,,,,,,,,,
2025-06-07,105615.626309,3.836503e+10,1.987610e+07,NaN,NaN,NaN,122.859,21862.5,1.087614e+13,1.935492e+14,3.125,413.0
2025-06-08,105793.650228,3.662623e+10,1.987611e+07,NaN,NaN,NaN,122.859,21862.5,1.087614e+13,1.935492e+14,3.125,414.0
2025-06-09,110294.099106,5.590319e+10,1.987695e+07,NaN,NaN,NaN,122.859,21862.5,1.087614e+13,1.935492e+14,3.125,415.0
2025-06-10,110257.237712,5.470010e+10,1.987801e+07,NaN,NaN,NaN,122.859,21862.5,1.087614e+13,1.935492e+14,3.125,416.0
2025-06-11,108686.627682,5.084266e+10,1.987749e+07,NaN,NaN,NaN,122.859,21862.5,1.087614e+13,1.935492e+14,3.125,417.0


In [32]:
# Ensure Main_df exists and index is DatetimeIndex
Main_df.index = pd.to_datetime(Main_df.index)
Main_df = Main_df.sort_index()

# 1) Halving dates and rewards
halving_dates = [
    pd.Timestamp('2012-11-28'),
    pd.Timestamp('2016-07-09'),
    pd.Timestamp('2020-05-11'),
    pd.Timestamp('2024-04-20')
]
rewards = [50, 25, 12.5, 6.25, 3.125]  # intervals: before 1st, 1st-2nd, etc.

# 2) Determine interval index for each date
idxs = np.searchsorted(halving_dates, Main_df.index, side='right')

# 3) Assign reward
Main_df['reward'] = [rewards[i] for i in idxs]

# 4) Compute last halving date for each row
last_halving = [halving_dates[i-1] if i > 0 else pd.NaT for i in idxs]
last_halving_series = pd.Series(last_halving, index=Main_df.index)

# 5) Days since last halving using .dt.days
delta = Main_df.index.to_series() - last_halving_series
Main_df['days_since_halving'] = delta.dt.days

# 6) Inspect
Main_df[['days_since_halving', 'reward']].tail()


,days_since_halving,reward
Date,,
2025-06-07,413.0,3.125
2025-06-08,414.0,3.125
2025-06-09,415.0,3.125
2025-06-10,416.0,3.125
2025-06-11,417.0,3.125


In [33]:

# 1) Define your date range (match your Main_df index)
start = Main_df.index.min().strftime("%Y-%m-%d")
end   = Main_df.index.max().strftime("%Y-%m-%d")

# 2) Fetch the daily Fed Funds Effective Rate
fed_rate = web.DataReader('DFF', 'fred', start, end)

# 3) Rename column for clarity
fed_rate.columns = ['fed_funds_rate']

# 4) (Optional) If you want monthly instead, use:
#    fed_rate = web.DataReader('FEDFUNDS', 'fred', start, end)
#    fed_rate.columns = ['fed_funds_rate_monthly']

# 5) Merge into Main_df (on the Date index)
Main_df = Main_df.sort_index()
Main_df = Main_df.merge(fed_rate, how='left', left_index=True, right_index=True)

# 6) Fill any missing days via forward‐fill (Fed rate usually constant until next change)
Main_df['fed_funds_rate'] = Main_df['fed_funds_rate'].ffill()

# Inspect
Main_df[['fed_funds_rate']].head()
Main_df.tail()

,BTC,BTC_Volume,BTC_count,ETH,ETH_Volume,ETH_count,dxy,US_M2,EA_M2,CN_M2,reward,days_since_halving,fed_funds_rate
2025-06-07,105615.626309,3.836503e+10,1.987610e+07,NaN,NaN,NaN,122.859,21862.5,1.087614e+13,1.935492e+14,3.125,413.0,4.33
2025-06-08,105793.650228,3.662623e+10,1.987611e+07,NaN,NaN,NaN,122.859,21862.5,1.087614e+13,1.935492e+14,3.125,414.0,4.33
2025-06-09,110294.099106,5.590319e+10,1.987695e+07,NaN,NaN,NaN,122.859,21862.5,1.087614e+13,1.935492e+14,3.125,415.0,4.33
2025-06-10,110257.237712,5.470010e+10,1.987801e+07,NaN,NaN,NaN,122.859,21862.5,1.087614e+13,1.935492e+14,3.125,416.0,4.33
2025-06-11,108686.627682,5.084266e+10,1.987749e+07,NaN,NaN,NaN,122.859,21862.5,1.087614e+13,1.935492e+14,3.125,417.0,4.33


In [14]:
import pandas as pd
import pandas_datareader.data as web

start, end = "2010-01-01", "2025-05-04"
gold_fred = web.DataReader("GOLDAMGBD228NLBM", "fred", start, end)

# Rename for clarity
gold_fred.columns = ["gold_fix_usd_ounce"]

gold_fred.head()


RemoteDataError: Unable to read URL: https://fred.stlouisfed.org/graph/fredgraph.csv?id=GOLDAMGBD228NLBM
Response Text:
b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n    <meta name="viewport" content="width=device-width, initial-scale=1">\r\n    <title>Error - St. Louis Fed</title>\r\n    <meta name="description" content="">\r\n    <meta name="keywords" content="">    \r\n    <link rel="stylesheet" type="text/css" href="/assets/bootstrap/dist/css/bootstrap.min.css">\r\n    <link rel="stylesheet" type="text/css" href="/css/home.min.css?1553087253">\r\n    <link rel="stylesheet" type="text/css" href="/assets/fontawesome-free/css/all.min.css">\r\n    <link rel="stylesheet" type="text/css" href="/assets/select2/dist/css/select2.min.css">\r\n    <style>p {\r\n        margin-bottom: 1.5em;\r\n    }</style>\r\n</head>\r\n<body>\r\n<link rel="preconnect" href="https://fonts.googleapis.com">\n<link r

In [34]:
Main_df.to_csv("../Data/Main_df.csv", index=True)